# The Knapsack problem

The [**0-1 knapsack problem**](https://en.wikipedia.org/wiki/Knapsack_problem) : Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

The model:

Variables: $X_i$, $i=1,2,3,...n$ the number of items of each kind

Constraints: 
$\sum_{i=1}^n w_i X_i \leq W$
where $w_i$ is the weight of the $i$ object and $W$ is the weight limit.

Objective:
$\max \sum_{i=1}^n v_i X_i$
where $v_i$ is the value of the $i$ object

In [1]:
# Knapshak with a genetic algorithm
library(GA)
library(lpSolveAPI)
library(slam)

v = c(6, 5, 8, 9, 6, 7, 3)
w = c(2, 3, 6, 7, 5, 9, 4)
W = 9


knapsack <- function(v,w,W, bigM) {
    function(x){
     f = x %*% v
   penalty = bigM * abs((x %*% w)-W)
   return(f - penalty)
   }   
}
   

set.seed(1234)
GA <- ga(type = "binary", fitness = knapsack(v,w,W, bigM=sum(w)), nBits = length(w),
         maxiter = 1000, run = 200, popSize = 20)

summary(GA)

Loading required package: foreach
Loading required package: iterators
Package 'GA' version 3.2.2
Type 'citation("GA")' for citing this R package in publications.

Attaching package: ‘GA’

The following object is masked from ‘package:utils’:

    de



── Genetic Algorithm ─────────────────── 

GA settings: 
Type                  =  binary 
Population size       =  20 
Number of generations =  1000 
Elitism               =  1 
Crossover probability =  0.8 
Mutation probability  =  0.1 

GA results: 
Iterations             = 235 
Fitness function value = 15 
Solution = 
     x1 x2 x3 x4 x5 x6 x7
[1,]  1  0  0  1  0  0  0

GA returns also real valued solution, which is much faster. We can convert it to binary, this however does not guaraty always a solution to the initial problem

In [2]:
nVar = length(w)

decode2 <- function(n)
{ 
    function(x){
      x = floor(x)
  out <- structure(x, names = paste0('x',1:n))
  return(out)  
    }
    
  
}

fitness2 <- function(v,w,W, bigM) 
{ 
    function(x){
     x <- decode2(length(w))(x)
     f = x %*% v
     penalty = bigM * abs((x %*% w)-W) #sum(w) is big_M
     return(f - penalty   )
    }
}

GA2 <- ga(type = "real-valued", fitness = fitness2(v,w,W, bigM=sum(w)), 
          lower = rep(0,nVar), upper = rep(2,nVar),
          popSize = 20, maxiter = 1000, run = 200)

summary(GA2)

t(apply(GA2@solution, 1, decode2(nVar)))


── Genetic Algorithm ─────────────────── 

GA settings: 
Type                  =  real-valued 
Population size       =  20 
Number of generations =  1000 
Elitism               =  1 
Crossover probability =  0.8 
Mutation probability  =  0.1 
Search domain = 
      x1 x2 x3 x4 x5 x6 x7
lower  0  0  0  0  0  0  0
upper  2  2  2  2  2  2  2

GA results: 
Iterations             = 200 
Fitness function value = 15 
Solutions = 
            x1        x2        x3       x4        x5        x6        x7
[1,]  1.502270 0.5912053 0.5945475 1.323297 0.5085275 0.5900299 0.5163094
[2,]  1.522136 0.5832152 0.5933516 1.332589 0.5083571 0.5803317 0.6056592
[3,]  1.498147 0.6062452 0.5764401 1.148006 0.5081540 0.5773775 0.6805300
[4,]  1.836665 0.6299451 0.5959994 1.333580 0.5084753 0.5774148 0.5693409
[5,]  1.608877 0.6332093 0.5946958 1.333679 0.5088023 0.5795786 0.5390614
[6,]  1.499840 0.5334812 0.5766536 1.297765 0.5081222 0.5773890 0.6397314
[7,]  1.574694 0.5120252 0.5940075 1.324016 0.5086014 0

x1,x2,x3,x4,x5,x6,x7
1,0,0,1,0,0,0
1,0,0,1,0,0,0
1,0,0,1,0,0,0
1,0,0,1,0,0,0
1,0,0,1,0,0,0
1,0,0,1,0,0,0
1,0,0,1,0,0,0
1,0,0,1,0,0,0
1,0,0,1,0,0,0
1,0,0,1,0,0,0


In [3]:
ga_binary = function(v,w,W, bigM){
    GA <- ga(type = "binary", fitness = knapsack(v,w,W, bigM), nBits = length(w),
         maxiter = 1000, run = 200, popSize = 20)

    #print(GA)
    return(list(GA@solution,
               GA@fitnessValue))
}

ga_continuous = function(v,w,W, bigM){
    GA <- ga(type = "real-valued", fitness = fitness2(v,w,W, bigM), 
          lower = rep(0,length(w)), upper = rep(2,length(w)),
          popSize = 20, maxiter = 1000, run = 200)

    #print(GA)
    return(list(t(apply(GA@solution, 1, decode2(length(w)))),
               GA@fitnessValue))
}


## Compare speed with lpsolve

In [4]:
lp_opt = function(v,w,W){
    nVar = length(v)
    lprec <- make.lp(1,nVar)
    
    set.row(lprec, 1, w,
          indices = 1:nVar)
    
    set.constr.value(lprec, lhs = c(NA), rhs = W, 1)
    set.bounds(lprec, lower = rep(0, nVar), upper = rep(1, nVar), columns = 1:nVar)
    set.type(lprec,columns = 1:nVar, type = c("integer"))
    lp.control(lprec, sense = "max")
    set.objfn(lprec, v)
    solve.lpExtPtr(lprec)
    sols <- get.variables(lprec)
    maximize <- get.objective(lprec)
    
    return(list(sols, maximize))
}

In [5]:
lp_opt(v,w,W)

[[1]]
[1] 1 0 0 1 0 0 0

[[2]]
[1] 15

In [6]:
n = 5000
v1 = sample.int(n, n)
w1 = sample.int(n, n)
W1 = sum(w1)*2/3
cat("number of variables: ", n ,"rhs: " , W1)

number of variables:  5000 rhs:  8335000

In [7]:
start.time <- Sys.time()

res_lopt = lp_opt(v1,w1,W1)

end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

res_lopt
cat("constraint satisfied: ", res_lopt[[1]] %*% w1 <= W1)

Time difference of 48.27027 secs

[[1]]
   [1] 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 0 1 1
  [38] 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 1 1
  [75] 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1
 [112] 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 0 0 1 1 1 1
 [149] 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 [186] 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1
 [223] 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1
 [260] 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1
 [297] 0 0 0 1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0
 [334] 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1
 [371] 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 [408] 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1
 [445] 1 0 1 1 0 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 0
 [482] 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1
 [519] 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0 1 1 0 0 0 1 1 1 1
 [556] 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1
 [593] 1 1 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0
 [630] 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0
 [667] 0 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 0 1
 [704] 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 1 0 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1
 [741] 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0
 [778] 1 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1 1
 [815] 0 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1 1 1 0
 [852] 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1
 [889] 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1
 [926] 0 1 1 1 1 1 0 1 1 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 [963] 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[1000] 0 1 1 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 0 1 0 1 1 1
[1037] 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0
[1074] 0 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1 0 1 1 1 1 1 1
[1111] 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1
[1148] 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1 1
[1185] 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1
[1222] 1 1 1 1 1 0 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 0
[1259] 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 0 0 0 0 1 1
[1296] 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 0 1 1 0 1 0 1 1 1
[1333] 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1
[1370] 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 1 0 0 1 1 0 1 1 1 0 0 1 0 1
[1407] 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 0
[1444] 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1
[1481] 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 1
[1518] 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0
[1555] 1 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 0 0 1 1 1
[1592] 1 1 0 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1
[1629] 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1
[1666] 0 0 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 1 0 1 1 1 1
[1703] 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1
[1740] 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1
[1777] 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1
[1814] 0 1 1 0 0 1 1 0 1 

constraint satisfied:  TRUE

In [8]:
start.time <- Sys.time()

res_bin = ga_binary(v1,w1,W1, sum(w1))

end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

res_bin

cat("contraint missed by: ",res_bin[[1]] %*% w1 - W1, "\n")

cat("optimal value found: ", res_bin[[1]] %*% v1)

Time difference of 6.801825 secs

x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x4991,x4992,x4993,x4994,x4995,x4996,x4997,x4998,x4999,x5000
1,0,1,1,1,0,0,0,0,1,...,0,0,0,0,1,1,1,1,1,1


contraint missed by:  -1097562 
optimal value found:  7023795

In [9]:
start.time <- Sys.time()

res_cont = ga_continuous(v1,w1,W1, sum(w1))

end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

res_cont
nrows_mat = dim(res_cont[[1]])[1]
cat("contraint missed by: ",res_cont[[1]][nrows_mat,] %*% w1 - W1,"\n")
cat("optimal value found: ", res_cont[[1]][nrows_mat,] %*% v1)

Time difference of 29.59848 secs

x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x4991,x4992,x4993,x4994,x4995,x4996,x4997,x4998,x4999,x5000
1,1,1,1,1,1,0,1,1,0,...,1,0,1,0,0,0,1,1,0,0


contraint missed by:  -878036 
optimal value found:  7185448